In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('nc_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_10_max', 'new_deaths_10_mean', 'current_intensive_care_patients', 'rainfall_mm', 'minimum_temperature_celsius', 'maximum_temperature_celsius', 'new_deaths_50_std', 'average_temperature_celsius', 'relative_humidity', 'new_cases_10_min', 'new_cases_50_max', 'new_deaths_10_std', 'new_cases_100_mean', 'dew_point', 'new_deaths_50_mean', 'Series_Complete_5Plus', 'stringency_index', 'Booster_Doses_65Plus', 'booster_rate', 'Booster_Doses', 'new_cases_50_mean', 'Administered_Dose1_Recip_5Plus', 'new_cases_100_std', 'Administered_Dose1_Recip_18Plus', 'new_deaths_100_mean', 'Series_Complete_18Plus']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_10_max', 'new_deaths_10_mean', 'maximum_temperature_celsius', 'rainfall_mm', 'current_intensive_care_patients', 'relative_humidity', 'new_deaths_10_std', 'dew_point', 'minimum_temperature_celsius', 'average_temperature_celsius', 'new_cases_50_std', 'new_deaths_50_std', 'new_deaths_50_mean', 'new_cases_50_mean', 'new_cases_100_std', 'Booster_Doses', 'Administered_Dose1_Recip', 'new_cases_10_min', 'new_cases_100_mean', 'Administered_Dose1_Recip_5Plus', 'Booster_Doses_50Plus', 'Booster_Doses_65Plus', 'new_deaths_100_std', 'new_deaths_10_max', 'new_deaths_100_mean', 'stringency_index']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_10_max', 'new_deaths_10_mean', 'new_hospitalized_patients', 'minimum_temperature_celsius', 'rainfall_mm', 'current_intensive_care_patients', 'maximum_temperature_celsius', 'new_cases_100_std', 'dew_point', 'Administered_Dose1_Recip_65Plus', 'new_deaths_100_max', 'new_deaths_100_mean', 'new_cases_50_std', 'new_deaths_50_mean', 'relative_humidity', 'new_cases_50_mean', 'new_deaths_10_std', 'new_cases_100_mean', 'Booster_Doses_50Plus', 'new_cases_10_min', 'Series_Complete_5Plus', 'stringency_index', 'Booster_Doses', 'new_deaths_50_min', 'new_deaths_50_std', 'new_deaths_100_std', 'Series_Complete_Yes']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'current_intensive_care_patients', 'new_cases_10_mean', 'minimum_temperature_celsius', 'new_hospitalized_patients', 'new_cases_10_max', 'new_deaths_10_mean', 'maximum_temperature_celsius', 'new_deaths_10_std', 'rainfall_mm', 'vaccination_rate_1dose', 'average_temperature_celsius', 'relative_humidity', 'new_deaths_100_mean', 'new_cases_50_std', 'new_deaths_50_mean', 'booster_rate', 'new_cases_50_mean', 'new_cases_100_mean', 'Administered_Dose1_Recip', 'Booster_Doses', 'dew_point', 'Administered_Dose1_Recip_12Plus', 'Series_Complete_18Plus', 'new_deaths_50_std', 'new_deaths_100_std', 'new_cases_100_std', 'Series_Complete_5Plus', 'vaccination_rate_complete']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_10_max', 'new_hospitalized_patients', 'current_intensive_care_patients', 'new_deaths_10_mean', 'minimum_temperature_celsius', 'new_deaths_10_std', 'maximum_temperature_celsius', 'rainfall_mm', 'Administered_Dose1_Recip_65Plus', 'average_temperature_celsius', 'Series_Complete_5Plus', 'new_cases_100_std', 'Series_Complete_18Plus', 'dew_point', 'new_cases_50_std', 'new_deaths_50_mean', 'new_deaths_100_mean', 'new_deaths_50_std', 'new_cases_100_mean', 'new_cases_50_mean', 'new_deaths_100_std', 'Booster_Doses_65Plus', 'relative_humidity', 'booster_rate', 'new_cases_100_max', 'Booster_Doses_18Plus', 'Booster_Doses_50Plus']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
---------------------------------------------------------------------
Basis Function                                  Pruned  Coefficient  
---------------------------------------------------------------------
(Intercept)                                     No      15979.7      
PEOPLE_DEATH_NEW_COUNT                          Yes     None         
h(new_cases_10_max-23742)                       No      -9.06734     
h(23742-new_cases_10_max)                       Yes     None         
h(new_deaths_10_std-58.7636)                    Yes     None         
h(58.7636-new_deaths_10_std)                    Yes     None         
h(new_cases_10_mean-5070.2)                     No      1.26424      
h(5070.2-new_cases_10_mean)                     No      -0.885513    
new_deaths_10_mean                              No      -108.291     
h(current_intensive_care_patients-960)          No      422.477      
h(960-current_intensive_care_patients)          Yes     None         
h(curren

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_hospitalized_patients', 'new_deaths_10_std', 'new_deaths_50_mean', 'relative_humidity', 'new_cases_50_mean', 'new_cases_100_std', 'new_cases_10_std', 'new_deaths_50_std', 'new_cases_100_mean', 'new_deaths_100_mean', 'maximum_temperature_celsius', 'new_deaths_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_mean', 'minimum_temperature_celsius', 'new_cases_10_max', 'rainfall_mm', 'current_intensive_care_patients', 'dew_point'}


## XGBoost Feature Selection, includes lag and rolling

In [14]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [15]:
covid_df = pd.read_csv('nc_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [16]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['PEOPLE_DEATH_NEW_COUNT', 'new_cases_7_lag', 'new_cases_10_mean', 'new_cases_10_std', 'new_cases_1 lag', 'new_deaths_1_lag', 'new_hospitalized_patients', 'current_intensive_care_patients', 'new_cases_10_max', 'new_cases_3_lag', 'new_cases_100_max', 'new_cases_30_lag', 'new_deaths_50_std', 'new_cases_50_max', 'new_cases_4_lag', 'new_cases_100_mean', 'new_deaths_7_lag', 'minimum_temperature_celsius', 'maximum_temperature_celsius', 'Booster_Doses_50Plus', 'rainfall_mm', 'new_cases_50_lag', 'new_deaths_2_lag', 'new_deaths_4_lag', 'new_deaths_50_mean', 'Administered_Dose1_Recip_12Plus', 'new_deaths_3_lag', 'Booster_Doses', 'new_deaths_10_std', 'vaccination_rate_complete', 'new_deaths_30_lag', 'vaccination_rate_1dose', 'new_cases_2_lag', 'new_cases_50_std', 'Series_Complete_18Plus', 'booster_rate', 'dew_point', 'average_temperature_celsius', 'new_deaths_10_mean', 'new_cases_150_lag']


In [17]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_7_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_1 lag', 'new_cases_3_lag', 'new_cases_2_lag', 'new_cases_30_lag', 'new_hospitalized_patients', 'new_deaths_3_lag', 'new_deaths_1_lag', 'relative_humidity', 'new_deaths_4_lag', 'current_intensive_care_patients', 'new_deaths_30_lag', 'new_deaths_7_lag', 'rainfall_mm', 'new_deaths_50_lag', 'new_deaths_2_lag', 'average_temperature_celsius', 'minimum_temperature_celsius', 'maximum_temperature_celsius', 'new_cases_150_lag', 'new_deaths_10_mean', 'new_cases_4_lag', 'Administered_Dose1_Recip', 'new_deaths_10_std', 'new_cases_100_std', 'new_cases_50_std']


In [18]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_7_lag', 'new_cases_10_max', 'facial_coverings', 'new_cases_10_mean', 'new_cases_10_std', 'new_deaths_1_lag', 'PEOPLE_DEATH_NEW_COUNT', 'rainfall_mm', 'new_deaths_50_std', 'new_deaths_50_lag', 'new_hospitalized_patients', 'new_cases_3_lag', 'new_cases_1 lag', 'minimum_temperature_celsius', 'new_cases_100_mean', 'new_cases_2_lag', 'new_deaths_100_std', 'Booster_Doses_65Plus', 'current_intensive_care_patients', 'new_deaths_10_std', 'Administered_Dose1_Recip_5Plus', 'new_deaths_7_lag', 'new_cases_50_std', 'new_deaths_50_min', 'new_deaths_100_max', 'average_temperature_celsius', 'Administered_Dose1_Recip', 'new_deaths_50_mean', 'new_deaths_10_mean', 'new_cases_100_std']


In [19]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
------------------------------------------------------------------------
Basis Function                                     Pruned  Coefficient  
------------------------------------------------------------------------
(Intercept)                                        No      -64597.6     
PEOPLE_DEATH_NEW_COUNT                             No      26.2575      
h(new_cases_10_max-23742)                          No      1.06409      
h(23742-new_cases_10_max)                          Yes     None         
present(new_cases_7_lag)                           Yes     None         
missing(new_cases_7_lag)                           Yes     None         
h(new_cases_7_lag-5532)*present(new_cases_7_lag)   No      -0.171864    
h(5532-new_cases_7_lag)*present(new_cases_7_lag)   Yes     None         
present(new_deaths_7_lag)                          Yes     None         
missing(new_deaths_7_lag)                          No      -5238.07     
h(new_deaths_7_lag-122)*present(new_dea

In [20]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [21]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [22]:
full = common_member(a, b, c)
full

{'new_deaths_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_deaths_10_std', 'new_cases_10_mean', 'minimum_temperature_celsius', 'new_deaths_7_lag', 'new_cases_10_max', 'current_intensive_care_patients', 'new_cases_50_std', 'rainfall_mm', 'average_temperature_celsius', 'new_cases_1 lag', 'new_deaths_1_lag', 'new_cases_3_lag', 'new_cases_7_lag', 'new_cases_10_std', 'new_cases_2_lag'}
